# A League of Legends Competitive Analysis

**Name(s)**: Kevin Wong, Andrew Yang

**Website Link**: https://kev374k.github.io/lol_competitive_analysis/

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Baseline Model

In [2]:
df = pd.read_csv("2022_LoL_esports_match_data_from_OraclesElixir.csv")
df

/Users/kevinwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (2,76) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149395,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149396,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149397,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149398,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# cleaning data: converts columns 'datacompleteness', 'playoffs', 'result'
# we then get the columns that are needed for our hypothesis
# Code is from our league of legends competitive analysis

cleaned_data = df.assign(
    datacompleteness = df['datacompleteness'].apply(lambda x: True if x == "complete" else False),
    playoffs = df['playoffs'].apply(lambda x: True if x == 1 else False),
    result = df['result'].apply(lambda x: True if x == 1 else False)
    ).loc[
        :, ['result', 'gameid', 'datacompleteness', 'playoffs', 'side', 'position', 'kills', 'deaths', 'assists', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'champion', 'gamelength', 'cspm', 'dragons', 'barons', 'dpm', 'wpm', 'vspm']
    ].fillna(0)

In [35]:
cleaned_data = cleaned_data[cleaned_data['datacompleteness'] == True]

In [36]:
cleaned_data.head(12)

,result,gameid,datacompleteness,playoffs,side,position,kills,deaths,assists,doublekills,...,quadrakills,pentakills,champion,gamelength,cspm,dragons,barons,dpm,wpm,vspm
0,False,ESPORTSTMNT01_2690210,True,False,Blue,top,2,3,2,0.0,...,0.0,0.0,Renekton,1713,8.0911,0.0,0.0,552.2942,0.2802,0.9107
1,False,ESPORTSTMNT01_2690210,True,False,Blue,jng,2,5,6,0.0,...,0.0,0.0,Xin Zhao,1713,5.1839,0.0,0.0,412.0841,0.2102,1.6813
2,False,ESPORTSTMNT01_2690210,True,False,Blue,mid,2,2,3,0.0,...,0.0,0.0,LeBlanc,1713,6.7601,0.0,0.0,499.4046,0.6655,1.0158
3,False,ESPORTSTMNT01_2690210,True,False,Blue,bot,2,4,2,0.0,...,0.0,0.0,Samira,1713,7.9159,0.0,0.0,389.0018,0.4203,0.8757
4,False,ESPORTSTMNT01_2690210,True,False,Blue,sup,1,5,6,0.0,...,0.0,0.0,Leona,1713,1.4711,0.0,0.0,128.3012,1.0158,2.4168
5,True,ESPORTSTMNT01_2690210,True,False,Red,top,1,1,12,0.0,...,0.0,0.0,Gragas,1713,8.0210,0.0,0.0,611.3835,0.5254,1.0508
6,True,ESPORTSTMNT01_2690210,True,False,Red,jng,4,1,10,1.0,...,0.0,0.0,Viego,1713,6.4098,0.0,0.0,370.0175,0.2802,1.3660
7,True,ESPORTSTMNT01_2690210,True,False,Red,mid,6,3,12,2.0,...,0.0,0.0,Viktor,1713,7.5657,0.0,0.0,724.6935,0.5254,1.0158
8,True,ESPORTSTMNT01_2690210,True,False,Red,bot,8,2,10,3.0,...,0.0,0.0,Jinx,1713,11.1734,0.0,0.0,934.7461,0.3152,1.4011
9,True,ESPORTSTMNT01_2690210,True,False,Red,sup,0,2,18,0.0,...,0.0,0.0,Alistar,1713,1.0158,0.0,0.0,158.1786,1.6112,2.3468


### Baseline Model

Train a “baseline model” for your prediction task that uses at least two features. (For this requirement, two features means selecting at least two columns from your original dataset that you should transform). You can leave numerical features as-is, but you’ll need to take care of categorical columns using an appropriate encoding. Implement all steps (feature transforms and model training) in a single sklearn Pipeline.

Note: Both now and in Final Model Step, make sure to evaluate your model’s ability to generalize to unseen data!

There is no “required” performance metric that your baseline model needs to achieve.

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer


In [48]:
train_X, test_X, train_y, test_y = train_test_split(cleaned_data, cleaned_data['result'])

In [49]:
column_transformer = ColumnTransformer([
    ('side', OneHotEncoder(), ['side']),
    ('kills', FunctionTransformer(), ['kills']),
    ('deaths', FunctionTransformer(), ['deaths']),
    ('assists', FunctionTransformer(), ['assists']),
    ('cspm', FunctionTransformer(), ['cspm']),
    ('dragons', FunctionTransformer(), ['dragons']),
    ('barons', FunctionTransformer(), ['barons'])
])

pipeline = Pipeline([
    ('columns', column_transformer),
    ('log-reg', LogisticRegression()),
])

In [51]:
pipeline.fit(train_X, train_y)
predictions = pipeline.predict(test_X)

In [52]:
baseline_performance = np.mean(predictions == test_y)
baseline_performance

0.8846383929977016

### Final Model

### Assessment of Missingness

In [17]:
# To assess missingness, we want to analyze what is actually missing
missing_data = pd.DataFrame(df.isnull().sum()).T
missing_data.iloc[:, 40:50]

,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs
0,128138,124500,124500,128226,128226,128138,128138,128138,128138,128226


In [18]:
# We've seen that a lot of out data is missing, especially near the end of the data; specifically, 
# things like 'opp_csat15', 'golddiffat15', and more are missing. In this case, we want to test if there's
# a specific reason why this data may be missing; 
missing_important_data = df[df['opp_csat15'].isnull()]

# Here, we want to get rid of unnecessary columns that won't be needed for both a permutation test and
# one's that have mostly unique columns, because otherwise we won't be able to compare distributions easily
missing_important_data = missing_important_data.loc[:, ['league', 'split', 'playoffs', 'patch', 'position']]
missing_important_data

,league,split,playoffs,patch,position
24,LPL,Spring,0,12.01,top
25,LPL,Spring,0,12.01,jng
26,LPL,Spring,0,12.01,mid
27,LPL,Spring,0,12.01,bot
28,LPL,Spring,0,12.01,sup
...,...,...,...,...,...
149395,DCup,NaN,0,12.23,mid
149396,DCup,NaN,0,12.23,bot
149397,DCup,NaN,0,12.23,sup
149398,DCup,NaN,0,12.23,team


In [28]:
# we are going to find what to test what type of data is missing when analyzing the different 
league_dependence = missing_important_data.groupby('league')['playoffs'].count()
split_dependence = missing_important_data.groupby('split')['playoffs'].count()
playoff_depedence = missing_important_data.groupby('playoffs')['playoffs'].count()
patch_dependence = missing_important_data.groupby('patch')['playoffs'].count()
position_dependence = missing_important_data.groupby('position')['playoffs'].count()

# while looking through these, we can see some stark observations
# for 'league_dependence', although we have 49 leagues, only 4 show up here
# for 'split_dependence', we had a relatively even amount of missing for each split
# for 'playoff_depedence', most were non-playoff games, but that may be an indication of the amount of playoff games vs normal games
# for 'patch_dependence', it was all over the place
for data in [league_dependence]:#, split_dependence, playoff_depedence, patch_dependence, position_dependence]:
    fig = px.bar(data/data.sum(), title = f"League Missingness Distribution on 'opp_csat15'")
    fig.show()
    fig.write_html('docs/assets/league_missingness_distribution.html')


In [20]:
# Now, let's do a permutation test on the overall dataframe to see if the missing values have the same distribution as this
# If they don't then it is likely that this column and the other columns are MAR 
test_df = (df
           .assign(league = np.random.permutation(df['league']))
           .assign(split = np.random.permutation(df['split']))
           .assign(playoffs = np.random.permutation(df['playoffs']))
           .assign(patch = np.random.permutation(df['patch']))
           .assign(position = np.random.permutation(df['position']))
           )
           
test_df = test_df[test_df['opp_csat15'].isnull()].loc[:, ['league', 'split', 'playoffs', 'patch', 'position']]
test_df

,league,split,playoffs,patch,position
24,LCK,Split 1,1,12.05,bot
25,LCKC,Summer,1,12.04,sup
26,SL (LATAM),NaN,0,12.03,jng
27,LCO,Split 2,0,12.09,team
28,NEXO,Spring,0,12.04,sup
...,...,...,...,...,...
149395,CBLOLA,Summer,1,12.05,jng
149396,NLC,Summer,0,12.03,mid
149397,PGC,Spring,0,12.05,top
149398,VCS,Summer,1,12.02,bot


In [26]:
league_dependence = test_df.groupby('league')['playoffs'].count()
split_dependence = test_df.groupby('split')['playoffs'].count()
playoff_depedence = test_df.groupby('playoffs')['playoffs'].count()
patch_dependence = test_df.groupby('patch')['playoffs'].count()
position_dependence = test_df.groupby('position')['playoffs'].count()

for data in [league_dependence]: #, split_dependence, playoff_depedence, patch_dependence, position_dependence]:
    fig = px.bar(data/data.sum(), title = "Permutated League Missingness Distribution on 'opp_csat15'")
    fig.show()
    fig.write_html("docs/assets/permutated_league_missingness.html")

In [16]:
test_df = df.assign(league = np.random.permutation)
stats = test_df[test_df['opp_csat15'].isnull()].groupby('league')['playoffs'].count()

In [17]:
# just through observing the graphs, the most stark difference comes from the regions, where it seems like LPL
# and LDL carry the most missing values (both are from China). In order to test this, let's compare the highest 
# in our filtered data set (LDL, 0.517867)

# In this case, these are our hypotheses (testing at the 0.01 significance level)
# Null Hypothesis: Data that is missing comes from the same 'league' distribution as all other data
# Alternative Hypothesis: Data that is missing is significantly more likely to be one 'league' than others

differences = []
for _ in range(1000):
    test_df = df.assign(league = np.random.permutation(df['league']))
    stats = test_df[test_df['opp_csat15'].isnull()].groupby('league')['playoffs'].count()
    differences.append(stats[stats.index == 'LDL'].iloc[0]/stats.sum())
    
p_value = np.mean(np.array(differences) >= 0.517867)

In [18]:
# Since our p-value is 0, this means that the missing of the 'opp_csat15' is reliant on the 'league' column,
# particularly when testing with the LDL and LPL teams (Chinese). These possibly suggest that there is an issue
# with getting Chinese Data on these columns, or that Chinese teams don't record this data

p_value

0.0

In [19]:
# Now for testing something else that might have a different missingness, we want to see if there's anything that 
# could reasonably be different based on randomness. If we look at the number of times missingness was determined
# based on the playoffs, the highest total percentage was 0.8801539. Let's test this with a null and alternative hypotheses

# Null Hypothesis: The missingness of NA values in the 'opp_csat15' column comes from the same distribution regardless of the position player's have played
# Alternative Hypothesis: The missingness of NA values in 'opp_csat15' column is dependent on a position played

position_differences = []
for _ in range(1000):
    test_df = df.assign(position = np.random.permutation(df['position']))
    stats = test_df[test_df['opp_csat15'].isnull()].groupby('position')['gameid'].count()
    position_differences.append(stats.iloc[0]/stats.sum())

position_pval = np.mean(np.array(position_differences) >= (1/6))

In [35]:
test_df = df.assign(position = np.random.permutation(df['position']))
stats = test_df[test_df['opp_csat15'].isnull()].groupby('position')['gameid'].count()
fig = px.bar(stats/stats.sum(), title = "Permutated Position Missingness Distribution on 'opp_csat15'")
fig.show()
fig.write_html("docs/assets/permutated_position_missingness.html")


In [20]:
# Since our p-value is over 0.01 (our significance level), we can accept our null hypothesis and state that
# the missingness of column 'opp_csat15' is not dependent on the position being played by players at the time

position_pval

0.512

### Hypothesis Testing

In [21]:
# We need to operationalize what winning means
# First we can add columns for KDA((Kills + Assists) / Deaths) as well as killparticipation

# def kda_calc(df):
#     return (df['kills'] + df['assists']) / max(df['deaths'], 1)
# def killpart_calc(df):
#     if df['teamkills'] == 0:
#         return 0
#     return (df['kills'] + df['assists']) / df['teamkills']

def winning_top(df):
    if df['xpdiffat15'] > 0:
        return True
    elif df['xpdiffat15'] < 0:
        return False
    return np.NaN

df = (filtered_df
#       .assign(kda = filtered_df.apply(kda_calc, axis=1))
#       .assign(killpart = filtered_df.apply(killpart_calc, axis=1))
      .assign(winlane = filtered_df.apply(winning_top, axis=1))
     )
df

,gameid,datacompleteness,playoffs,side,position,champion,gamelength,result,killsat15,assistsat15,deathsat15,goldat15,xpat15,csat15,golddiffat15,xpdiffat15,csdiffat15,winlane
0,ESPORTSTMNT01_2690210,True,False,Blue,top,Renekton,1713,False,0.0,1.0,0.0,5025.0,7560.0,135.0,391.0,345.0,14.0,True
5,ESPORTSTMNT01_2690210,True,False,Red,top,Gragas,1713,True,0.0,1.0,0.0,4634.0,7215.0,121.0,-391.0,-345.0,-14.0,False
12,ESPORTSTMNT01_2690219,True,False,Blue,top,Gragas,2114,False,0.0,1.0,1.0,4673.0,7020.0,110.0,-1484.0,-652.0,-30.0,False
17,ESPORTSTMNT01_2690219,True,False,Red,top,Gangplank,2114,True,1.0,0.0,1.0,6157.0,7672.0,140.0,1484.0,652.0,30.0,True
36,ESPORTSTMNT01_2690227,True,False,Blue,top,Renekton,1972,True,2.0,0.0,0.0,5856.0,7759.0,146.0,904.0,85.0,4.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149105,ESPORTSTMNT01_3268686,True,False,Red,top,K'Sante,2325,True,0.0,1.0,2.0,4782.0,7650.0,122.0,-801.0,829.0,11.0,True
149112,ESPORTSTMNT01_3269631,True,False,Blue,top,Fiora,2076,True,2.0,1.0,1.0,7262.0,8025.0,137.0,2728.0,1467.0,32.0,True
149117,ESPORTSTMNT01_3269631,True,False,Red,top,K'Sante,2076,False,0.0,1.0,3.0,4534.0,6558.0,105.0,-2728.0,-1467.0,-32.0,False
149124,ESPORTSTMNT01_3268705,True,False,Blue,top,Gangplank,1680,True,2.0,1.0,2.0,6485.0,7618.0,123.0,1537.0,645.0,15.0,True


In [22]:
df['winlane'].unique()

array([True, False, nan], dtype=object)

In [26]:
perm_df = df[~df['winlane'].isna()]
perm_df

,gameid,datacompleteness,playoffs,side,position,champion,gamelength,result,killsat15,assistsat15,deathsat15,goldat15,xpat15,csat15,golddiffat15,xpdiffat15,csdiffat15,winlane
0,ESPORTSTMNT01_2690210,True,False,Blue,top,Renekton,1713,False,0.0,1.0,0.0,5025.0,7560.0,135.0,391.0,345.0,14.0,True
5,ESPORTSTMNT01_2690210,True,False,Red,top,Gragas,1713,True,0.0,1.0,0.0,4634.0,7215.0,121.0,-391.0,-345.0,-14.0,False
12,ESPORTSTMNT01_2690219,True,False,Blue,top,Gragas,2114,False,0.0,1.0,1.0,4673.0,7020.0,110.0,-1484.0,-652.0,-30.0,False
17,ESPORTSTMNT01_2690219,True,False,Red,top,Gangplank,2114,True,1.0,0.0,1.0,6157.0,7672.0,140.0,1484.0,652.0,30.0,True
36,ESPORTSTMNT01_2690227,True,False,Blue,top,Renekton,1972,True,2.0,0.0,0.0,5856.0,7759.0,146.0,904.0,85.0,4.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149105,ESPORTSTMNT01_3268686,True,False,Red,top,K'Sante,2325,True,0.0,1.0,2.0,4782.0,7650.0,122.0,-801.0,829.0,11.0,True
149112,ESPORTSTMNT01_3269631,True,False,Blue,top,Fiora,2076,True,2.0,1.0,1.0,7262.0,8025.0,137.0,2728.0,1467.0,32.0,True
149117,ESPORTSTMNT01_3269631,True,False,Red,top,K'Sante,2076,False,0.0,1.0,3.0,4534.0,6558.0,105.0,-2728.0,-1467.0,-32.0,False
149124,ESPORTSTMNT01_3268705,True,False,Blue,top,Gangplank,1680,True,2.0,1.0,2.0,6485.0,7618.0,123.0,1537.0,645.0,15.0,True


In [29]:
perm_df[perm_df['winlane'] == False]['result'].mean()

0.39754253308128545

In [38]:
#Performing a permutation test using diff of means as test statistic
def test_stat(df):
    return df[df['winlane'] == True]['result'].mean() - df[df['winlane'] == False]['result'].mean()

def simulate_stat(df):
    shuffed_df = df.assign(winlane = np.random.permutation(df['winlane']))
    return test_stat(shuffed_df)

observed_stat = test_stat(perm_df)

simulated_stats = []
for i in range(10_000):
    simulated_stats.append(simulate_stat(perm_df))

np.mean(np.array(simulated_stats) > observed_stat)

0.0

In [ ]:
###Cleaned up code for permutation test
#Function to assign which top laner in a game had the advantage.
    #Assign True if the top laner had the advantage, False if at disadvantage
    #Assign np.NaN if neither player had advantage
def winning_top(df):
    if df['xpdiffat15'] > 0:
        return True
    elif df['xpdiffat15'] < 0:
        return False
    return np.NaN

#Create df with column based on winning or losing top lane
winlane_df = (filtered_df
      .assign(winlane = filtered_df.apply(winning_top, axis=1))
     )
#Remove rows with no decisive winning lane
perm_df = df[~df['winlane'].isna()]
perm_df

#Function to calculate test statistic given dataframe
    #Uses diff of means
    #Proportion of team win for winning top lanes - proportion of team lose for losing top lanes
def test_stat(df):
    return df[df['winlane'] == True]['result'].mean() - df[df['winlane'] == False]['result'].mean()

#Function to shuffle the winlane column and return a new test_stat
def simulate_stat(df):
    shuffed_df = df.assign(winlane = np.random.permutation(df['winlane']))
    return test_stat(shuffed_df)

#Calculate observed statistic, the test stat based on the unshuffled column
observed_stat = test_stat(perm_df)

#simulate 10,000 stats and calculate a significance value
simulated_stats = []
for i in range(10_000):
    simulated_stats.append(simulate_stat(perm_df))

np.mean(np.array(simulated_stats) > observed_stat)